In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
import os
!ls 
os.chdir('gdrive/My Drive/dl20/final_project/FinalProject')

!ls

gdrive	sample_data
'=1.4'		  explore_the_data.ipynb   objectDectionFastRCNN.ipynb
'=2.0.8'	  helper.py		   __pycache__
 data		  Mask_RCNN		   requirements.txt
 data_helper.py   MaskRCNN.ipynb
 env		  model_loader.py


In [4]:
import os
from PIL import Image

import random

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
matplotlib.rcParams['figure.figsize'] = [5, 5]
matplotlib.rcParams['figure.dpi'] = 200

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

from data_helper import UnlabeledDataset, LabeledDataset, FastRCNNLabeledDataset
from helper import collate_fn, draw_box

import pickle
import time
import copy

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Test an example code

In [5]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
images, boxes = torch.rand(4, 3, 600, 1200), torch.rand(4, 11, 4)
labels = torch.randint(1, 91, (4, 11))
images = list(image for image in images)
targets = []
for i in range(len(images)):
    d = {}
    d['boxes'] = boxes[i]
    d['labels'] = labels[i]
    targets.append(d)
output = model(images, targets)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


In [6]:
output

{'loss_box_reg': tensor(0.0024, grad_fn=<DivBackward0>),
 'loss_classifier': tensor(0.0786, grad_fn=<NllLossBackward>),
 'loss_objectness': tensor(13.6640, grad_fn=<BinaryCrossEntropyWithLogitsBackward>),
 'loss_rpn_box_reg': tensor(nan, grad_fn=<DivBackward0>)}

In [0]:
# All the images are saved in image_folder
# All the labels are saved in the annotation_csv file

# image_folder = '../data'
# annotation_csv = '../data/annotation.csv'

image_folder = 'data'
annotation_csv = 'data/annotation.csv'
labeled_scene_index = np.arange(133, 134)

# image_folder = '/Users/nhungle/Downloads/dl20_data'
# annotation_csv = '/Users/nhungle/Downloads/dl20_data/annotation.csv'

In [0]:
cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda else "cpu")

# Labeled dataset

In [0]:
def inspect_target(index):
    NUM_SAMPLE_PER_SCENE = 126
    NUM_IMAGE_PER_SAMPLE = 6
    image_names = [
        'CAM_FRONT_LEFT.jpeg',
        'CAM_FRONT.jpeg',
        'CAM_FRONT_RIGHT.jpeg',
        'CAM_BACK_LEFT.jpeg',
        'CAM_BACK.jpeg',
        'CAM_BACK_RIGHT.jpeg',
        ]
    scene_index = labeled_scene_index 
    scene_id = scene_index[index // NUM_SAMPLE_PER_SCENE]
    sample_id = index % NUM_SAMPLE_PER_SCENE
    sample_path = os.path.join(image_folder, f'scene_{scene_id}', f'sample_{sample_id}') 
    images = []
    for image_name in image_names:
        image_path = os.path.join(sample_path, image_name)
        image = Image.open(image_path)
        images.append(transform(image))
    image_tensor = torch.stack(images)
    annotation_file = annotation_csv 
    annotation_dataframe = pd.read_csv(annotation_file)
    data_entries = annotation_dataframe[(annotation_dataframe['scene'] == scene_id) & (annotation_dataframe['sample'] == sample_id)]
    corners = data_entries[['fl_x', 'fr_x', 'bl_x', 'br_x', 'fl_y', 'fr_y','bl_y', 'br_y']].to_numpy()
    categories = data_entries.category_id.to_numpy()
    num_objects = len(categories)
    boxes = []
    for i in range(num_objects):
        xmin = min(corners[i][:4])
        xmax = max(corners[i][:4])
        ymin = min(corners[i][4:])
        ymax = max(corners[i][4:])
        boxes.append([xmin, ymin, xmax, ymax])
    return data_entries, image_tensor

In [0]:
# The labeled dataset can only be retrieved by sample.
# And all the returned data are tuple of tensors, since bounding boxes may have different size
# You can choose whether the loader returns the extra_info. It is optional. You don't have to use it.
transform = torchvision.transforms.ToTensor()
labeled_trainset = LabeledDataset(image_folder=image_folder,
                                  annotation_file=annotation_csv,
                                  scene_index=labeled_scene_index,
                                  transform=transform,
                                  extra_info=True
                                 )
fasterRCNN_trainset = FastRCNNLabeledDataset(image_folder=image_folder,
                                  annotation_file=annotation_csv,
                                  scene_index=labeled_scene_index,
                                  transform=transform,
                                  extra_info=True
                                 )
trainloader = torch.utils.data.DataLoader(fasterRCNN_trainset,
                                          batch_size=1,
                                          shuffle=True,
                                          num_workers=2, collate_fn=collate_fn)

In [11]:
sample, target, road_image, extra = iter(trainloader).next()
print(torch.stack(sample).shape)
sample = torch.stack(sample)

torch.Size([1, 6, 3, 256, 306])


In [12]:
index = target[0]['image_id'].item()
data_entries, idx_tensor = inspect_target(index)
data_entries["category_id"].values == target[0]['labels'].data.numpy()

array([ True,  True,  True,  True,  True,  True,  True,  True])

In [0]:
feature_extractor = torchvision.models.resnet18(pretrained=False)
feature_extractor = nn.Sequential(*list(feature_extractor.children())[:-2])
#feature_extractor.to(device)
for param in feature_extractor.parameters():
    param.requires_grad = True

In [0]:
def concat_features(features, dim = 2):
    #dim 0 ==> stacking the images in the channel dimension
    #dim 1 ==> stacking the images in row dimension
    #dim 2 ==> stacking the images in column dimension
    tensor_tuples = torch.unbind(features, dim=0)
    concatenated_fm = torch.cat(tensor_tuples, dim=dim)
    return concatenated_fm 

## Change this to work for higher batch size while combining mutliple samples

In [0]:
def prepare_inputs(sample):
    """
    Input: samples is a cuda tensor with size [batch_size, 6, 3, 256, 306]
    Output: a list of batch_size tensor, each tensor with size [512, 16, 114]
    """
    batch_size = sample.shape[0]
    extracted_fm = []
    for i in range(len(sample)):
        features = feature_extractor(sample[i])
        print(features.shape)
        features = concat_features(features)
        print(features.shape)
        extracted_fm.append(features)
    #print(len(extracted_fm))
    return extracted_fm


In [16]:
inputs = prepare_inputs(sample)

torch.Size([6, 512, 8, 10])
torch.Size([512, 8, 60])


In [0]:
#inputs 

In [18]:
inputs[0].shape

torch.Size([512, 8, 60])

## Fast RCNN

In [0]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

In [0]:
def customize_VGG16():
    model = torchvision.models.vgg16(pretrained=True)
    
    features = list(model.features)[:30]
    classifier = model.classifier
    
    classifier = list(classifier)
    # delete the Linear layer
    del classifier[6]
    classifier = nn.Sequential(*classifier)

    #freeze top4 conv layer
    for layer in features[:10]:
        for p in layer.parameters():
            p.requires_grad = False
    features = nn.Sequential(*features)
        
    return features, classifier

In [21]:
backbone, box_head = customize_VGG16()

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth


In [0]:
backbone.out_channels = 512
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                           aspect_ratios=((0.5, 1.0, 2.0),))
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=[0],
                                                output_size=7,
                                                sampling_ratio=2)

In [0]:
pretrained_model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

In [0]:
model = torchvision.models.detection.faster_rcnn.FasterRCNN(
            backbone, 
            rpn_anchor_generator = anchor_generator,
            box_roi_pool = roi_pooler,
            box_head = box_head,
            num_classes = 9
            )
#             box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(4096, num_classes=9))

## Test my own model on the right set of inputs

In [0]:
output1 = pretrained_model(images, targets)

In [28]:
output1

{'loss_box_reg': tensor(0.0036, grad_fn=<DivBackward0>),
 'loss_classifier': tensor(0.1347, grad_fn=<NllLossBackward>),
 'loss_objectness': tensor(13.7790, grad_fn=<BinaryCrossEntropyWithLogitsBackward>),
 'loss_rpn_box_reg': tensor(nan, grad_fn=<DivBackward0>)}

In [29]:
output

{'loss_box_reg': tensor(0.0037, grad_fn=<DivBackward0>),
 'loss_classifier': tensor(0.1321, grad_fn=<NllLossBackward>),
 'loss_objectness': tensor(13.7630, grad_fn=<BinaryCrossEntropyWithLogitsBackward>),
 'loss_rpn_box_reg': tensor(nan, grad_fn=<DivBackward0>)}

## Test the pretrained model on my inputs and targets

In [26]:
# Inspect inputs and target before bringing into model
train_inputs = [image.view(3, 512, 160) for image in inputs]
train_targets = list(target)
print(len(train_inputs))
print(train_inputs[0].shape)
print(len(train_targets))
print(train_targets[0].keys())

1
torch.Size([3, 512, 160])
1
dict_keys(['boxes', 'labels', 'image_id', 'iscrowd', 'bounding_box'])


In [0]:
output2 = pretrained_model(train_inputs, train_targets)

In [29]:
output2

{'loss_box_reg': tensor(0.0121, grad_fn=<DivBackward0>),
 'loss_classifier': tensor(0.3394, grad_fn=<NllLossBackward>),
 'loss_objectness': tensor(0.6961, grad_fn=<BinaryCrossEntropyWithLogitsBackward>),
 'loss_rpn_box_reg': tensor(0.4735, grad_fn=<DivBackward0>)}